In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import gradio as gr

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
openai = OpenAI()


In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

In [ ]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_llama(prompt):
    llama_model = "llama3.2"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    # Use the stream parameter in ollama.chat
    stream = ollama.chat(
        model=llama_model, 
        messages=messages, 
        stream=True
    )
    
    result = ""
    for chunk in stream:
        # Check if the chunk contains content
        if chunk['message']['content']:
            result += chunk['message']['content']
            yield result

In [ ]:

view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [ ]:
view = gr.Interface(
    fn=stream_llama,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [18]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Llama":
        result = stream_llama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [19]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Llama"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [21]:
gr.close_all()

Closing server running on port: 7864
Closing server running on port: 7863
Closing server running on port: 7860
Closing server running on port: 7862
